In [1]:
import numpy as np
import sympy
import csv
from json import loads

lab_number = '2.2.1'
var_number = 24


def toCopyString(matrix) -> str:
    return 'matrix(' + str(matrix).replace('Matrix([', '[').replace('])', ']')[1:-1].replace(' ', '') + ')'

def fromCopyString(string: str):
    return loads(string.replace('matrix(', '[').replace(')', ']'))

def получить_матрицы_варианта(number):
    filename = f'./{lab_number}_variants.txt'
    with open(filename, 'r') as file:
        variants = csv.reader(file, delimiter=',')
        variant = list(variants)[number - 1][1].split(';')
        A = fromCopyString(variant[0])
        B = fromCopyString(variant[1])
        L = fromCopyString(variant[2])
    return A, B, L

In [2]:
A, B, L = получить_матрицы_варианта(var_number)
A = sympy.Matrix(A)
B = sympy.Matrix(B)
L = sympy.Matrix(L)
L

Matrix([
[ 0,  0],
[-1,  2],
[-1, -2]])

In [3]:
def матрица_управляемости(A, B, L):
    матрица = sympy.zeros(3, 3)
    матрица[:, 0] = B 
    матрица[:, 1] = A*B
    матрица[:, 2] = A**2*B
    return матрица

In [4]:
S = матрица_управляемости(A, B, L)
S

Matrix([
[-2,   4, -17],
[-1,  -1,  -7],
[10, -23,  55]])

In [5]:
toCopyString(S)

'matrix([-2,4,-17],[-1,-1,-7],[10,-23,55])'

In [6]:
def определитель_матрицы_управляемости(S):
    определитель = S.det()
    # print('Да!' if определитель != 0 else 'Нет :(')
    return определитель

In [7]:
print(определитель_матрицы_управляемости(S))

-189


In [8]:
def матрица_C(L):
    res = sympy.zeros(2, 3)
    if L[0, 0] or L[0, 1]:
        res[0, 0] = 1
        if L[1, 0] or L[1, 1]:
            res[1, 1] = 1
        else:
            res[1, 2] = 1
    elif L[1, 0] or L[1, 1]:
        res[0, 1] = 1
        if L[2, 0] or L[2, 1]:
            res[1, 2] = 1
    elif L[2, 0] or L[2, 1]:
        res[0, 2] = 1
    
    return res

In [9]:
C = матрица_C(L)
C

Matrix([
[0, 1, 0],
[0, 0, 1]])

In [10]:
toCopyString(C)

'matrix([0,1,0],[0,0,1])'

In [11]:
def вектор_d(A, B, L):
    s = sympy.symbols('s')
    tmp = (s*sympy.eye(3) - A).inv()*B
    tmp = tmp.applyfunc(lambda x: sympy.simplify(x))
    highest_denominator = sympy.fraction(tmp[0, 0])[1]
    for i in range(3):
        if not sympy.div(sympy.fraction(tmp[i, 0])[1], highest_denominator)[1]:
            highest_denominator = sympy.fraction(tmp[i, 0])[1]
    # print(highest_denominator)
    return sympy.Matrix(sympy.Poly(highest_denominator).all_coeffs()[:0:-1])

In [12]:
d = вектор_d(A, B, L)
d

Matrix([
[-36],
[-24],
[ -1]])

In [13]:
def матрица_g(A, B, L):
    s = sympy.symbols('s')
    tmp = (s*sympy.eye(3) - A).inv()*B
    tmp = tmp.applyfunc(lambda x: sympy.simplify(x))
    # multiply every element in tmp by denominator of tmp[0, 0]
    tmp = tmp.applyfunc(lambda x: sympy.simplify(x*sympy.fraction(tmp[0, 0])[1]))
    tmp = tmp.applyfunc(lambda x: sympy.simplify(x*sympy.fraction(tmp[1, 0])[1]))
    tmp = tmp.applyfunc(lambda x: sympy.simplify(x*sympy.fraction(tmp[2, 0])[1]))
    return tmp

In [14]:
g = матрица_g(A, B, L)
g

Matrix([
[  -2*s**2 + 6*s + 27],
[           18 - s**2],
[10*s**2 - 33*s - 162]])

In [15]:
def матрица_L(A, B, L):
    s = sympy.symbols('s')
    tmp = матрица_g(A, B, L)
    res = sympy.zeros(3, 3)
    res[0] = sympy.Poly(tmp[0, 0], s).all_coeffs()[::-1]
    res[1] = sympy.Poly(tmp[1, 0], s).all_coeffs()[::-1]
    res[2] = sympy.Poly(tmp[2, 0], s).all_coeffs()[::-1]
    return res.T

In [16]:
def matrix_L_(A, B, L):
    B0 = sympy.eye(3)
    f1 = -sympy.trace(A)
    g1 = B.copy()
    B1 = B0*A + f1*sympy.eye(3)
    f2 = -sympy.trace(B1*A) / 2
    g2 = B1*B
    B2 = B1*A + f2*sympy.eye(3)
    f3 = -sympy.trace(B2*A) / 3
    g3 = B2*B

    d = sympy.Matrix([f3, f2, f1])
    L = sympy.Matrix([g3.T, g2.T, g1.T])
    return L.T

In [17]:
def vector_d(A, B, L):
    B0 = sympy.eye(3)
    f1 = -sympy.trace(A)
    B1 = B0*A + f1*sympy.eye(3)
    f2 = -sympy.trace(B1*A) / 2
    B2 = B1*A + f2*sympy.eye(3)
    f3 = -sympy.trace(B2*A) / 3

    d = sympy.Matrix([f3, f2, f1])
    return d

In [18]:
d = vector_d(A, B, L)
d

Matrix([
[-36],
[-24],
[ -1]])

In [19]:
matrix_L = матрица_L(A, B, L)
matrix_L

Matrix([
[  27,   6, -2],
[  18,   0, -1],
[-162, -33, 10]])

In [20]:
matrix_L_1 = matrix_L_(A, B, L)
matrix_L_1

Matrix([
[  27,   6, -2],
[  18,   0, -1],
[-162, -33, 10]])

In [21]:
toCopyString(matrix_L)
# [::-1]

'matrix([27,6,-2],[18,0,-1],[-162,-33,10])'

In [22]:
toCopyString(matrix_L_1)

'matrix([27,6,-2],[18,0,-1],[-162,-33,10])'

In [23]:
def матрица_H(A, B, L):
    s = sympy.symbols('s')
    C = матрица_C(L)
    tmp = (s*sympy.eye(3) - A + B)

$ \alpha \vec{a} = \beta $

$ \alpha = (I_H - E_H E_B^{-1}I_B)(L^T)^{-1} $

In [42]:
def матрица_ограничения(A, B, L):
    res = sympy.zeros(3, 3)
    d = vector_d(A, B, L)
    b = sympy.zeros(3, 1)
    lambdas = sympy.Matrix([L[i, 0] + L[i, 1]*sympy.I for i in range(3)])
    is_first = True
    for i, lmbd in enumerate(lambdas):
        if lmbd:
            if lmbd.is_real:
                res[i] = sympy.Matrix([1, lmbd, lmbd**2])
                b[i] = -lmbd**3
            else:
                print(lmbd)
                
                if sympy.im(lmbd) < 0:
                    lmbd = lmbd.conjugate()
                angle = sympy.arg(lmbd).n()
                module = sympy.sqrt(lmbd*sympy.conjugate(lmbd)).n()
                print(module)
                print(angle)
                print(sympy.cos(angle))
                print(sympy.sin(angle))

                if angle < 0:
                    angle += sympy.pi
                if is_first:
                    res[i] = sympy.Matrix([1, module*sympy.cos(angle), module**2*sympy.cos(2*angle)])
                    b[i] = -module**3*sympy.cos(3*angle)
                    is_first = False
                else:
                    res[i] = sympy.Matrix([0, module*sympy.sin(angle), module**2*sympy.sin(2*angle)])
                    b[i] = -module**3*sympy.sin(3*angle)
        else:
            res[i, i] = 1
            b[i] = d[i]
    res = res.n().T
    
    a = res.inv()*b

    return (res, b.n(), a.n())


In [43]:
alpha, b, a = матрица_ограничения(A, B, L)
alpha

-1 + 2*I
2.23606797749979
2.03444393579570
-0.447213595499958
0.894427190999916
-1 - 2*I
2.23606797749979
2.03444393579570
-0.447213595499958
0.894427190999916


Matrix([
[1.0,    0,    0],
[1.0, -1.0, -3.0],
[  0,  2.0, -4.0]])

In [26]:
toCopyString(alpha)

'matrix([1.00000000000000,0,0],[1.00000000000000,-1.00000000000000,-3.00000000000000],[0,2.00000000000000,-4.00000000000000])'

In [27]:
b

Matrix([
[-36.0],
[-11.0],
[  2.0]])

In [28]:
toCopyString(b)

'matrix([-36.0000000000000],[-11.0000000000000],[2.00000000000000])'

In [29]:
a

Matrix([
[-36.0],
[ -9.4],
[ -5.2]])

In [30]:
toCopyString(a)

'matrix([-36.0000000000000],[-9.40000000000000],[-5.20000000000000])'

In [31]:
def матрица_Iu(A, B, L):
    res = sympy.zeros(2, 3)
    is_first = True
    for i in range(3):
        if L[i, 0] or L[i, 1]:
            if is_first:
                res[0, i] = 1
                is_first = False
            else:
                res[1, i] = 1
    return res

In [32]:
Iu = матрица_Iu(A, B, L)
Iu

Matrix([
[0, 1, 0],
[0, 0, 1]])

In [33]:
toCopyString(Iu)

'matrix([0,1,0],[0,0,1])'

In [34]:
def матрица_Cu(A, B, L):
    res = sympy.zeros(2, 2)
    C_T = матрица_C(L).T
    cur_row = 0
    for i in range(3):
        if C_T[i, 0] or C_T[i, 1]:
            res[cur_row, :] = C_T[i, :]
            cur_row += 1
    return res

In [35]:
Cu = матрица_Cu(A, B, L)
Cu

Matrix([
[1, 0],
[0, 1]])

In [36]:
toCopyString(Cu)

'matrix([1,0],[0,1])'

In [37]:
def матрица_p(A, B, L):
    matrix_L = matrix_L_(A, B, L)
    Iu = матрица_Iu(A, B, L)
    Cu = матрица_Cu(A, B, L)
    a = матрица_ограничения(A, B, L)[-1]
    d = vector_d(A, B, L)
    res = (Cu.T).inv()*Iu*(matrix_L.inv()).T*(a-d)
    return res

In [38]:
p = матрица_p(A, B, L)
p

Matrix([
[-2.37142857142857],
[  1.7047619047619]])

In [39]:
toCopyString(p)

'matrix([-2.37142857142857],[1.70476190476190])'

In [324]:
for var_number in range(24, 25):
    A, B, L = получить_матрицы_варианта(var_number)
    A = sympy.Matrix(A)
    B = sympy.Matrix(B)
    L = sympy.Matrix(L)
    try:
        S                          = матрица_управляемости(A, B, L)
        определитель               = определитель_матрицы_управляемости(S)
        матрица_C_                 = матрица_C(L)
        матрица_L_                 = matrix_L_(A, B, L)
        левая_матрица_ограничения,\
        правая_матрица_ограничения,\
        матрица_А                  = матрица_ограничения(A, B, L)
        Iu                         = матрица_Iu(A, B, L)
        Cu                         = матрица_Cu(A, B, L)
        p                          = матрица_p(A, B, L)

        print(var_number)
        with open(f'./results/{var_number}.txt', 'w') as file:
            file.write(toCopyString(S) + '\n')
            file.write(str(определитель) + '\n')
            file.write(toCopyString(матрица_C_) + '\n')
            file.write(toCopyString(матрица_L_) + '\n')
            file.write(toCopyString(левая_матрица_ограничения) + '\n')
            file.write(toCopyString(правая_матрица_ограничения) + '\n')
            file.write(toCopyString(матрица_А) + '\n')
            file.write(toCopyString(Iu) + '\n')
            file.write(toCopyString(Cu) + '\n')
            file.write(toCopyString(p) + '\n')
    # catch error
    except Exception as e:
        print(f'Error in {var_number}: {e}')   
        

20
21
22
23
24
25
26
27
28
29
30
